In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
from sklearn import preprocessing

from sys import path
path.append(r'E:\data_mining\project\IJCAI-17-口碑\koubei')
import base
import predict

In [2]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [3]:
train_data = pd.read_csv('../data/predict/fill_predict_data.txt', sep='\t', index_col=0, encoding='UTF-8')
train_data.columns = np.array(range(21))

In [4]:
mean_3_3_3_prediction = pd.read_csv('../data/predict/mean/3_3_3_prediction.txt', sep='\t', index_col=0, encoding='UTF-8')

In [5]:
arima_prediction = pd.read_csv('../data/predict/arima/arima_prediction.txt', sep='\t', index_col=0, encoding='UTF-8')
arima_prediction = predict.FixFinalPredict(arima_prediction)

In [6]:
def judgePredIntersectType(s):
    #不相交 且mean 大于arima
    if len(s[s >= 0]) == 14:
        return 1
    #不相交 且mean 小于arima
    elif len(s[s <= 0]) == 14:
        return 2
    #相交
    else:
        return 0

In [7]:
def judgeMeanIntersectType(s):
    #不相交 且mean 小于pred
    if len(s[s >= 0]) == 14:
        return 1
    #不相交 且mean 大于pred
    elif len(s[s <= 0]) == 14:
        return 2
    #相交
    else:
        return 0

In [8]:
train_df = pd.DataFrame()

In [9]:
intersects = mean_3_3_3_prediction - arima_prediction
intersects = intersects.apply(judgePredIntersectType, axis=1)
train_df['pred_intersect'] = intersects

In [10]:
train_df['pre_week_mean'] = train_data[np.array(range(14, 21))].mean(axis=1).astype(np.int)

In [11]:
intersects = arima_prediction.sub(train_df['pre_week_mean'].squeeze(), axis=0)
intersects = intersects.apply(judgeMeanIntersectType, axis=1)
train_df['mean_arima_intersect'] = intersects

In [12]:
intersects = mean_3_3_3_prediction.sub(train_df['pre_week_mean'].squeeze(), axis=0)
intersects = intersects.apply(judgeMeanIntersectType, axis=1)
train_df['mean_mean_intersect'] = intersects

In [13]:
X_train_data = train_data[np.array(range(21))]

In [14]:
X_train_df = pd.DataFrame()
X_train_df['max'] = X_train_data.max(axis=1)
X_train_df['min'] = X_train_data.min(axis=1)
X_train_df['std'] = X_train_data.std(axis=1)
X_train_df['var'] = X_train_data.var(axis=1)
X_train_df['mean'] = X_train_data.mean(axis=1)
X_train_df['median'] = X_train_data.median(axis=1)

In [15]:
X_train_df['mean_max'] = mean_3_3_3_prediction.max(axis=1)
X_train_df['mean_min'] = mean_3_3_3_prediction.min(axis=1)
X_train_df['mean_std'] = mean_3_3_3_prediction.std(axis=1)
X_train_df['mean_var'] = mean_3_3_3_prediction.var(axis=1)
X_train_df['mean_mean'] = mean_3_3_3_prediction.mean(axis=1)
X_train_df['mean_median'] = mean_3_3_3_prediction.median(axis=1)

X_train_df['arima_max'] = arima_prediction.max(axis=1)
X_train_df['arima_min'] = arima_prediction.min(axis=1)
X_train_df['arima_std'] = arima_prediction.std(axis=1)
X_train_df['arima_var'] = arima_prediction.var(axis=1)
X_train_df['arima_mean'] = arima_prediction.mean(axis=1)
X_train_df['arima_median'] = arima_prediction.median(axis=1)

In [16]:
X_train_df['diff_arima_max'] = arima_prediction.max(axis=1) - X_train_data.max(axis=1)
X_train_df['diff_arima_min'] = arima_prediction.min(axis=1) - X_train_data.min(axis=1)
X_train_df['diff_arima_std'] = arima_prediction.std(axis=1) - X_train_data.std(axis=1)
X_train_df['diff_arima_var'] = arima_prediction.var(axis=1)- X_train_data.var(axis=1)
X_train_df['diff_arima_mean'] = arima_prediction.mean(axis=1) - X_train_data.mean(axis=1)
X_train_df['diff_arima_median'] = arima_prediction.median(axis=1) - X_train_data.median(axis=1)

X_train_df['diff_mean_max'] = mean_3_3_3_prediction.max(axis=1) - X_train_data.max(axis=1)
X_train_df['diff_mean_min'] = mean_3_3_3_prediction.min(axis=1) - X_train_data.min(axis=1)
X_train_df['diff_mean_std'] = mean_3_3_3_prediction.std(axis=1) - X_train_data.std(axis=1)
X_train_df['diff_mean_var'] = mean_3_3_3_prediction.var(axis=1) - X_train_data.var(axis=1)
X_train_df['diff_mean_mean'] = mean_3_3_3_prediction.mean(axis=1) - X_train_data.mean(axis=1)
X_train_df['diff_mean_median'] = mean_3_3_3_prediction.median(axis=1) - X_train_data.median(axis=1)

X_train_df['diff_mean_arima_max'] = mean_3_3_3_prediction.max(axis=1) - arima_prediction.max(axis=1)
X_train_df['diff_mean_arima_min'] = mean_3_3_3_prediction.min(axis=1) - arima_prediction.min(axis=1)
X_train_df['diff_mean_arima_std'] = mean_3_3_3_prediction.std(axis=1) - arima_prediction.std(axis=1)
X_train_df['diff_mean_arima_var'] = mean_3_3_3_prediction.var(axis=1) - arima_prediction.var(axis=1)
X_train_df['diff_mean_arima_mean'] = mean_3_3_3_prediction.mean(axis=1) - arima_prediction.mean(axis=1)
X_train_df['diff_mean_arima_median'] = mean_3_3_3_prediction.median(axis=1) - arima_prediction.median(axis=1)

In [17]:
X_train_scaled = preprocessing.scale(X_train_df.values)
X_train_df = pd.DataFrame(X_train_scaled)
X_train_df = X_train_df.set_index(np.array(range(1, 2001)))

In [18]:
X_train_df = pd.merge(X_train_df,  pd.get_dummies(train_df['pred_intersect'], prefix='pred_intersect').astype(np.int), left_index=True, right_index=True)

In [19]:
X_train_df = pd.merge(X_train_df,  pd.get_dummies(train_df['mean_arima_intersect'], prefix='mean_arima_intersect').astype(np.int), left_index=True, right_index=True)

In [20]:
X_train_df = pd.merge(X_train_df,  pd.get_dummies(train_df['mean_mean_intersect'], prefix='mean_mean_intersect').astype(np.int), left_index=True, right_index=True)

In [21]:
today = datetime.date.today()

In [33]:
X_train_df.to_csv('../data/predict/mean_arima/X_test' + today.isoformat() + '.csv')